In [24]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import silhouette_score
import nltk
from keras.layers import LSTM, Dense, Embedding
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
poems = gutenberg.raw('blake-poems.txt')
stories = gutenberg.raw('bryant-stories.txt')
busterbrown = gutenberg.raw('burgess-busterbrown.txt')
alice = gutenberg.raw('carroll-alice.txt')
ball = gutenberg.raw('chesterton-ball.txt')
parents = gutenberg.raw('edgeworth-parents.txt')
moby_dick = gutenberg.raw('melville-moby_dick.txt')
paradise = gutenberg.raw('milton-paradise.txt')
hamlet = gutenberg.raw('shakespeare-hamlet.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
poems = re.sub(r'CHAPTER .*', '', poems)
stories = re.sub(r'CHAPTER .*', '', stories)
busterbrown = re.sub(r'CHAPTER .*', '', busterbrown)
ball = re.sub(r'CHAPTER .*', '', ball)
parents = re.sub(r'CHAPTER .*', '', parents)
moby_dick = re.sub(r'CHAPTER .*', '', moby_dick)
paradise = re.sub(r'CHAPTER .*', '', paradise)
hamlet = re.sub(r'CHAPTER .*', '', hamlet)

In [4]:
# Decrease size of our text dataset

alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
poems = text_cleaner(poems[:int(len(poems)/10)])
stories = text_cleaner(stories[:int(len(stories)/10)])
busterbrown = text_cleaner(busterbrown[:int(len(busterbrown)/10)])
ball = text_cleaner(ball[:int(len(ball)/10)])
parents = text_cleaner(parents[:int(len(parents)/10)])
moby_dick = text_cleaner(moby_dick[:int(len(moby_dick)/10)])
paradise = text_cleaner(paradise[:int(len(paradise)/10)])
hamlet = text_cleaner(hamlet[:int(len(hamlet)/10)])

In [5]:
nltk.sent_tokenize(alice)

["Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'",
 'So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 "There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!",
 'Oh dear!',
 "I shall be late!'",
 '(when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT-POCKET, and

In [6]:
# Convert all the paragraph into text and it's labels

alice_sents = [[sent, "Carroll"] for sent in nltk.sent_tokenize(alice)]
persuasion_sents = [[sent, "Austen"] for sent in nltk.sent_tokenize(persuasion)]
poems_sents = [[sent, "Blake"] for sent in nltk.sent_tokenize(poems)]
stories_sents = [[sent, "Bryant"] for sent in nltk.sent_tokenize(stories)]
busterbrown_sents = [[sent, "Burgess"] for sent in nltk.sent_tokenize(busterbrown)]
ball_sents = [[sent, "Chesterton"] for sent in nltk.sent_tokenize(ball)]
parents_sents = [[sent, "Edgeworth"] for sent in nltk.sent_tokenize(parents)]
moby_dick_sents = [[sent, "Melville"] for sent in nltk.sent_tokenize(moby_dick)]
paradise_sents = [[sent, "Milton"] for sent in nltk.sent_tokenize(paradise)]
hamlet_sents = [[sent, "Shakespeare"] for sent in nltk.sent_tokenize(hamlet)]

In [7]:
sentences = pd.DataFrame(alice_sents + persuasion_sents + poems_sents + stories_sents + busterbrown_sents + ball_sents + parents_sents + moby_dick_sents + paradise_sents + hamlet_sents)
sentences.head()

,0,1
0,Alice was beginning to get very tired of sitti...,Carroll
1,So she was considering in her own mind (as wel...,Carroll
2,There was nothing so VERY remarkable in that; ...,Carroll
3,Oh dear!,Carroll
4,I shall be late!',Carroll


In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences[0]).toarray()
Y = sentences[1]

In [9]:
X.shape

(3588, 9105)

## Creating clusters

In [10]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
y_predict = kmeans.predict(X)

In [11]:
print(silhouette_score(X, kmeans.labels_))
pd.crosstab(Y, y_predict)

0.009004489636521304


col_0,0,1,2,3,4,5,6,7,8,9
1,,,,,,,,,,
Austen,51,7,42,13,20,5,2,54,59,27
Blake,11,0,0,3,4,0,0,0,5,5
Bryant,88,51,12,15,18,3,4,35,59,42
Burgess,31,2,9,5,10,4,0,0,9,38
Carroll,38,9,14,4,0,3,4,41,4,1
Chesterton,61,28,34,71,46,14,2,0,119,64
Edgeworth,128,114,61,21,73,33,4,107,109,128
Melville,375,40,106,91,100,26,5,7,286,54
Milton,55,0,14,10,53,8,1,0,65,4


From cross tab poems from Blake is least identified in our clustering model. So our model performed pretty poorly.

In [12]:
sentences[1].value_counts()

Melville       1090
Edgeworth       778
Chesterton      439
Bryant          327
Austen          280
Milton          210
Shakespeare     210
Carroll         118
Burgess         108
Blake            28
Name: 1, dtype: int64

## Vectorizing methods

We will try TfidfVectorizer and CountVectorizer for converting our data into numerical form. Next we would compare both those models for higher accuracy.

In [39]:
X = sentences[0]
Y = sentences[1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [40]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, Y_train)

count_vectorizer = CountVectorizer()
X_count = count_vectorizer.fit_transform(X_train)
X_train_count, X_test_count, y_train, y_test = train_test_split(X_count, Y_train)

# Supervised Modeling

Now let's make some models with labels available to us. We'll try 3 different supervised models, RandomForestClassifier, GradientBoostingClassifier and LogisticRegression and see which works best.

### RandomForestClassifier

In [21]:
# random forest with tfidf vectorizer
rfc_tfidf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=30)
rfc_tfidf.fit(X_tfidf, Y)
print("Tfidf Train score: ", rfc_tfidf.score(X_train_tfidf, Y_train))
print("Tfidf Test score: ", rfc_tfidf.score(X_test_tfidf, Y_test))

# random forest with count vectorizer
rfc_count = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=30)
rfc_count.fit(X_count, Y)
print("Count Train score: ", rfc_count.score(X_train_count, Y_train))
print("Count Test score: ", rfc_count.score(X_test_count, Y_test))

Tfidf Train score:  0.30360460795243405
Tfidf Test score:  0.3032329988851728
Count Train score:  0.3240431066518023
Count Test score:  0.30992196209587514


### GradientBoostingClassifier

In [23]:
# Gradient boosting with tfidf vectorizer
gbc_tfidf = GradientBoostingClassifier()
gbc_tfidf.fit(X_tfidf, Y)
print("Tfidf Train score: ", gbc_tfidf.score(X_train_tfidf, Y_train))
print("Tfidf Test score: ", gbc_tfidf.score(X_test_tfidf, Y_test))

# Gradient boosting with count vectorizer
gbc_count = GradientBoostingClassifier()
gbc_count.fit(X_count, Y)
print("Count Train score: ", gbc_count.score(X_train_count, Y_train))
print("Count Test score: ", gbc_count.score(X_test_count, Y_test))

Tfidf Train score:  0.1958379784466741
Tfidf Test score:  0.20624303232998886
Count Train score:  0.8454106280193237
Count Test score:  0.8472686733556298


### LogisticRegression

In [24]:
# Logistic Regression with tfidf Vectorizer
lr_tfidf = LogisticRegression()
lr_tfidf.fit(X_tfidf, Y)
print("Tfidf Train score: ", lr_tfidf.score(X_tfidf, Y))
print("Tfidf Test score: ", lr_tfidf.score(X_test_tfidf, Y_test))

# Logistic Regression with count vectorizer
lr_count = LogisticRegression()
lr_count.fit(X_count, Y)
print("Count Train score: ", lr_count.score(X_train_count, Y_train))
print("Count Test score: ", lr_count.score(X_test_count, Y_test))

C:\Users\vivek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vivek\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Tfidf Train score:  0.8199554069119287
Tfidf Test score:  0.2129319955406912
Count Train score:  0.9535488665923448
Count Test score:  0.9509476031215162


Best model from supervised learning is Logistic Regression with accuracy of 95% in both train and test dataset. Best performing vectorizer is CountVectorizer in every model. TfidfVectorizer tends to overfitting in LogisticRegression.

# Unsupervise modeling

In [41]:
X_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [42]:
X_train = pad_sequences(X_train_count.toarray())

In [47]:
model = Sequential()
model.add(Embedding(4000, 128,input_length = X_train.shape[1], dropout=0.2))
model.add(LSTM(256, dropout_U=0.2, dropout_W=0.2, return_sequences=False))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 7736, 128)         512000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 906,497
Trainable params: 906,497
Non-trainable params: 0
_________________________________________________________________


In [44]:
y_train = pd.get_dummies(y_train)

In [48]:
history = model.fit(X_train, y_train, epochs=50, batch_size=50, validation_data=(X_test_count, y_test))

ValueError: Error when checking target: expected dense_4 to have shape (1,) but got array with shape (10,)

In [49]:
y_train

,Austen,Blake,Bryant,Burgess,Carroll,Chesterton,Edgeworth,Melville,Milton,Shakespeare
2174,0,0,0,0,0,0,0,1,0,0
1813,0,0,0,0,0,0,1,0,0,0
2156,0,0,0,0,0,0,0,1,0,0
495,0,0,1,0,0,0,0,0,0,0
234,1,0,0,0,0,0,0,0,0,0
3583,0,0,0,0,0,0,0,0,0,1
2577,0,0,0,0,0,0,0,1,0,0
983,0,0,0,0,0,1,0,0,0,0
3120,0,0,0,0,0,0,0,1,0,0
2825,0,0,0,0,0,0,0,1,0,0
